# convert fp16→fp8  float8_e4m3
量子化するスクリプトファイルです。  
Googleドライブをモデルファイルの保存先に使用します。実行時にアクセス許可を行ってください。  

## 操作手順  
1, メニューの「ランタイム」より「ランタイムのタイプを変更」し、ハードウェア アクセラレータ を「GPU」または「TPUv2」を選択してください。  
2, 全てのセルを実行してください。(Ctrl+F9)  

In [ ]:
#@title fp8変換ツール(float8_e4m3)
repo_id = "fal/AuraFlow-v0.3" #@param {type:"string"}
repo_filename = "aura_flow_0.3.safetensors" #@param {type:"string"}
save_dir = "/content/drive/MyDrive/ComfyUI/models/checkpoints" #@param {type:"string"}
save_filename = "aura_flow_0.3-fp8.safetensors" #@param {type:"string"}
is_debug = True #@param {type:"boolean"}

# 量子化を行わないレイヤー
non_quantized_layers = set(["vae.",
                            "model.double_layers.",
                            "model.single_layers.31"])

import json
from pathlib import Path
import time

from google.colab import drive, runtime
from safetensors.torch import load_file, save_file
import torch
from tqdm import tqdm
from huggingface_hub import hf_hub_download


def mount_drive(path: str):
  if path.startswith('/content/drive'):
    print("Google Driveをマウントしています。")
    drive.mount('/content/drive', force_remount=True)


def read_safetensors_metadata(file: str):
  """read safetensors metadata"""
  with open(file, 'rb') as f:
    header_size = int.from_bytes(f.read(8), 'little')
    header_json = f.read(header_size).decode('utf-8')
    header = json.loads(header_json)
    metadata = header.get('__metadata__', {})
    return metadata


try:
  # Googleドライブのマウント処理を行います。
  mount_drive(save_dir)
  # Repositoryからモデルファイルのダウンロードを行います。
  download_file = hf_hub_download(repo_id=repo_id, filename=repo_filename, revision="main")
  output_path : Path  = Path(save_dir) / save_filename
  output_path.parent.mkdir(parents=True, exist_ok=True)

  # メタ情報を読み取ります。
  metadata = read_safetensors_metadata(download_file)
  print(json.dumps(metadata, indent=4)) #show metadata

  sd_pruned = dict() #initialize empty dict
  layer_status = [] # レイヤー名とスキップ有無を保存するリスト
  state_dict = load_file(download_file) #load safetensors file
  # fp8化を行います。
  for key in tqdm(state_dict): #for each key in the safetensors file
    layer_name = str(key)
    # layer_nameがnon_quantized_layers内のいずれかのプレフィックスで始まるかをチェック
    if any(layer_name.startswith(skip_layer) for skip_layer in non_quantized_layers):
        sd_pruned[key] = state_dict[key]
        layer_status.append((layer_name, "スキップ"))  # スキップされた場合
        continue  # スキップする場合は次へ

    sd_pruned[key] = state_dict[key].to(torch.float8_e4m3fn)
    layer_status.append((layer_name, "処理済み"))  # スキップされなかった場合

  if is_debug:
    # レイヤー名とスキップの有無を出力
    for layer_name, status in layer_status:
      print(f"{layer_name}, {status}")

  #save_file(sd_pruned, output_path, metadata={"format": "pt", **metadata})
  print(download_file)
  print("ファイルを保存しました。Googleドライブ内の以下のフォルダを確認してください。")
  print(output_path)
finally:
    print("5秒後にランタイムの接続を解除します。")
    time.sleep(5)
    runtime.unassign()


Google Driveをマウントしています。
Mounted at /content/drive
{}


100%|██████████| 824/824 [01:11<00:00, 11.48it/s]


model.cond_seq_linear.weight, 処理済み
model.double_layers.0.attn.w1k.weight, スキップ
model.double_layers.0.attn.w1o.weight, スキップ
model.double_layers.0.attn.w1q.weight, スキップ
model.double_layers.0.attn.w1v.weight, スキップ
model.double_layers.0.attn.w2k.weight, スキップ
model.double_layers.0.attn.w2o.weight, スキップ
model.double_layers.0.attn.w2q.weight, スキップ
model.double_layers.0.attn.w2v.weight, スキップ
model.double_layers.0.mlpC.c_fc1.weight, スキップ
model.double_layers.0.mlpC.c_fc2.weight, スキップ
model.double_layers.0.mlpC.c_proj.weight, スキップ
model.double_layers.0.mlpX.c_fc1.weight, スキップ
model.double_layers.0.mlpX.c_fc2.weight, スキップ
model.double_layers.0.mlpX.c_proj.weight, スキップ
model.double_layers.0.modC.1.weight, スキップ
model.double_layers.0.modX.1.weight, スキップ
model.double_layers.1.attn.w1k.weight, スキップ
model.double_layers.1.attn.w1o.weight, スキップ
model.double_layers.1.attn.w1q.weight, スキップ
model.double_layers.1.attn.w1v.weight, スキップ
model.double_layers.1.attn.w2k.weight, スキップ
model.double_layers.1.attn.w2o.